please make these 2 files below first

In [1]:
#requirements.txt
"""accelerate==0.33.0
bitsandbytes==0.43.2
transformers==4.43.2
gradio
torch
python-dotenv"""
#Config.json
"""{""}"""

'{""}'

**Step 1: INSTALLING DEPENDENCIES**

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2

In [ ]:
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

**HF account configuration**

In [ ]:
config_data =json.load(open('Config.json'))
HF_TOKEN = config_data['HF_TOKEN']

In [ ]:
model_name ="meta-llama/Meta-Llama-3-8B"


Quantisation Configuration

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

**Loding the Tokenizer and the LLM**

In [ ]:
print(HF_TOKEN)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import os
if os.path.exists('requirements.txt'):
    # Read content from requirements.txt
    with open('requirements.txt', 'r') as f:
        requirements = f.read()
    # Split into lines and filter for bitsandbytes and triton
    lines = requirements.split('\n')
    # Filter for bitsandbytes and triton
    bitsandbytes_line = next((line for line in lines if 'bitsandbytes' in line), None)
    triton_line = next((line for line in lines if 'triton' in line), None)

    if bitsandbytes_line:
        # Extract version if present, otherwise just package name
        bitsandbytes_package = bitsandbytes_line.split('==')[0]
        print(f"Uninstalling {bitsandbytes_package}...")
        !pip uninstall -y {bitsandbytes_package}
    if triton_line:
        triton_package = triton_line.split('==')[0]
        print(f"Uninstalling {triton_package}...")
        !pip uninstall -y {triton_package}

# Now install the correct versions (or latest compatible)
print("Installing bitsandbytes and triton...")
!pip install bitsandbytes --upgrade
!pip install --pre triton --upgrade

Uninstalling bitsandbytes...
Found existing installation: bitsandbytes 0.43.2
Uninstalling bitsandbytes-0.43.2:
  Successfully uninstalled bitsandbytes-0.43.2
Installing bitsandbytes and triton...
  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl (59.4 MB)
  Using cached triton-3.5.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
Using cached triton-3.5.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (170.5 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.5.0
    Uninstalling triton-3.5.0:
      Successfully uninstalled triton-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.9.0+cu126 requires triton==3.5.0; platform_system == "Linux", but you have triton 3.5.1 which is inc

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
text_generator =pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

**Messages**

In [ ]:
#Note: end of 1st video
messages = [
    {
        "role":"system",
        "content":"You are a 'Indian Tax & Law Advisor' who helps Indian public with any Request or Advice"
    },
    {
        "role":"user",
        "content":"Who are you?"
    }
]

**Create Prompt**

In [ ]:
prompt = text_generator.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


In [ ]:
prompt

"<|im_start|>system\nYou are a 'Indian Tax & Law Advisor' who helps Indian public with any Request or Advice<|im_end|>\n<|im_start|>user\nWho are you?<|im_end|>\n<|im_start|>assistant\n"

In [ ]:
terminators = [
    text_generator.tokenizer.eos_token_id,
    text_generator.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = text_generator(
    prompt,
    max_new_tokens = 128,
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.6,
    top_p = 0.9,
)

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

Who are you?<|im_end|>
<|im_start|>sir
Who are you?<|im_end|>
<|im_start|>sir
Who are you?<|im_end|>
<|im_start|>sir
Who are you?<|im_end|>
<|im_start|>sir
Who are you?<|im_end|>
<|im_start|>sir
Who are you?<|im_end|>
<|im_start|>sir
Who are you?<|im_end|>
<|im_start|>sir
Who are


**Step 2: Build App**

In [ ]:
import gradio as gr

In [ ]:
def chat_function(message, history, system_prompt, max_new_tokens, temperature):
    messages = [{"role":"system","content":system_prompt},
                {"role":"user", "content":message}]
    prompt = text_generator.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,)
    terminators = [
        text_generator.tokenizer.eos_token_id,
        #text_generator.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
    outputs = text_generator(
        prompt,
        max_new_tokens = max_new_tokens,
        eos_token_id = terminators,
        do_sample = True,
        temperature = temperature + 0.1,
        top_p = 0.9,)
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
gr.ChatInterface(
    chat_function,
    textbox=gr.Textbox(placeholder="Enter message here", container=False, scale = 7),
    chatbot=gr.Chatbot(height=400),
    additional_inputs=[
        gr.Textbox("You are a 'Indian Tax & Law Advisor' who helps Indian public with any Request or Advice", label="System Prompt"),
        gr.Slider(500,4000, label="Max New Tokens"),
        gr.Slider(0,1, label="Temperature")
    ]
    ).launch()

/tmp/ipython-input-3411004550.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=400),
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:330: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c74344881cadeb6f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Task
Install the correct version of Triton (`3.5.0`) to resolve the compatibility issue with PyTorch, then re-attempt to load the model.

## Fix Triton Version

### Subtask:
Adjust the installation command to install the correct version of Triton (3.5.0) to resolve the compatibility issue with PyTorch.


**Reasoning**:
To resolve the compatibility issue, I will modify the specified code cell to uninstall existing `bitsandbytes` and `triton` versions and then install the required versions, `bitsandbytes==0.43.2` and `triton==3.5.0`.



In [ ]:
import os
if os.path.exists('requirements.txt'):
    # Read content from requirements.txt
    with open('requirements.txt', 'r') as f:
        requirements = f.read()
    # Split into lines and filter for bitsandbytes and triton
    lines = requirements.split('\n')
    # Filter for bitsandbytes and triton
    bitsandbytes_line = next((line for line in lines if 'bitsandbytes' in line), None)
    triton_line = next((line for line in lines if 'triton' in line), None)

    if bitsandbytes_line:
        # Extract version if present, otherwise just package name
        bitsandbytes_package = bitsandbytes_line.split('==')[0]
        print(f"Uninstalling {bitsandbytes_package}...")
        !pip uninstall -y {bitsandbytes_package}
    if triton_line:
        triton_package = triton_line.split('==')[0]
        print(f"Uninstalling {triton_package}...")
        !pip uninstall -y {triton_package}

# Now install the correct versions
print("Installing specific versions of bitsandbytes and triton...")
!pip install bitsandbytes==0.43.2
!pip install triton==3.5.0

Uninstalling bitsandbytes...
Found existing installation: bitsandbytes 0.43.2
Uninstalling bitsandbytes-0.43.2:
  Successfully uninstalled bitsandbytes-0.43.2
Installing specific versions of bitsandbytes and triton...
  Using cached bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
Using cached bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl (137.5 MB)
ERROR: Operation cancelled by user
^C
